# 코드 수정 2023 8월 2일 ~

`-` 이유 

* 겨울 프로젝트 당시에 결과를 빨리 내려고 코드를 너무 난잡하게 작성 및 다시 코드 리뷰를 하니 변경해야 할점이 있어 수정.

* 또한 프로젝트 끝난후 코드를 다시 수정하다가 뭔가를 잘못 했는지 모델이 전혀 예측을 하지못함-> 그 뒤로 수정을 하지 않음.

* 하지만 깃허브를 만든후에 공부겸 코드를 다시 수정해봐야겠다는 생각이 들어 수정하게 됨

# import

In [1]:
import plotly
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import matplotlib.pyplot as plt
import plotly.express as px

from scipy.stats import skew, kurtosis
import numpy as np
import pandas as pd
from glob import glob
import os


from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from torchmetrics import R2Score

import torchvision
import torch
import torchmetrics
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torch.utils.data import Dataset, DataLoader, random_split
import string
import torch.optim as optim

import torchmetrics
from torchmetrics import R2Score


from torchvision import datasets
from torch.utils.data import DataLoader
import torchvision.transforms as transforms
from tqdm.notebook import tqdm

import wandb
import math

C:\Users\31083\anaconda3\envs\test_envs\lib\site-packages\torchaudio\backend\utils.py:74: UserWarning: No audio backend is available.
  warnings.warn("No audio backend is available.")


In [6]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)


device_cpu = torch.device("cpu")
device_cpu

cuda:0


device(type='cpu')

In [3]:
r2score = R2Score().to(device)

### read

## NUM_FEATURES

In [ ]:
# feature 변수의 개수 지정
NUM_FEATURES = len(pc2_2.drop(' C1', 1).columns)
print(f'number of features: {NUM_FEATURES}')


number of features: 2


<ipython-input-21-eff78eb62edf>:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  NUM_FEATURES = len(pc2_2.drop(' C1', 1).columns)


# sweep

## 로그인

In [4]:
# Log in to your W&B account

wandb.login()

wandb: Currently logged in as: potatopotato (expect). Use `wandb login --relogin` to force relogin


True

In [5]:
sweep_config = {
    'method': 'bayes'
    }

metric = {
    'name': 'test_accuracy',
    'goal': 'maximize',
    'target': 0.99
    }

sweep_config['metric'] = metric

early_terminate = {'type': 'hyperband','max_iter': 40, 's': 2}


sweep_config['early_terminate'] = early_terminate

parameters_dict = {
    'optimizer': {
        'values': ['adam']
        },
    # 'fc_layer_size': {
    #     'values': [64,128, 256]
    #     },
    'dropout': {
          'values': [0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9]
        },
    }

sweep_config['parameters'] = parameters_dict





### read

## NUM_FEATURES

In [ ]:
# feature 변수의 개수 지정
NUM_FEATURES = len(pc2_2.drop(' C1', 1).columns)
print(f'number of features: {NUM_FEATURES}')


number of features: 2


<ipython-input-21-eff78eb62edf>:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  NUM_FEATURES = len(pc2_2.drop(' C1', 1).columns)


## 로그인

In [ ]:
# Log in to your W&B account

wandb.login()

wandb: Currently logged in as: potatopotato (expect). Use `wandb login --relogin` to force relogin


True

In [8]:
sweep_config = {
    'method': 'bayes'
    }

metric = {
    'name': 'test_accuracy',
    'goal': 'maximize',
    'target': 0.99
    }

sweep_config['metric'] = metric

early_terminate = {'type': 'hyperband','max_iter': 40, 's': 2}


sweep_config['early_terminate'] = early_terminate

parameters_dict = {
    'optimizer': {
        'values': ['adam']
        },
    'dropout': {
          'values': [0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9]
        },
    }

sweep_config['parameters'] = parameters_dict





In [9]:

parameters_dict.update({
    'learning_rate': {
        # a flat distribution between 0 and 0.1
        'distribution': 'uniform',
        'min': 0,
        'max': 0.1
      },
    'batch_size': {
        # integers between 32 and 256
        # with evenly-distributed logarithms 
        'distribution': 'q_log_uniform_values',
        'q': 8,
        'min': 64,
        'max': 2048,
      }

    })

In [10]:
parameters_dict.update({
    'epochs': {
        'value': 30
    }})

parameters_dict.update({
    'dropout': {
        'values': [0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9]
    }})

parameters_dict.update({
    'learning_rate': {'distribution': 'uniform', 'min': 0, 'max': 0.1},

    'batch_size': {'distribution': 'q_log_uniform_values',
   'q': 8,
   'min': 16,
   'max': 524}   
  })

In [11]:
pprint.pprint(sweep_config)

{'early_terminate': {'max_iter': 40, 's': 2, 'type': 'hyperband'},
 'method': 'bayes',
 'metric': {'goal': 'maximize', 'name': 'test_accuracy', 'target': 0.99},
 'parameters': {'batch_size': {'distribution': 'q_log_uniform_values',
                               'max': 524,
                               'min': 16,
                               'q': 8},
                'dropout': {'values': [0.1,
                                       0.2,
                                       0.3,
                                       0.4,
                                       0.5,
                                       0.6,
                                       0.7,
                                       0.8,
                                       0.9]},
                'epochs': {'value': 30},
                'learning_rate': {'distribution': 'uniform',
                                  'max': 0.1,
                                  'min': 0},
                'optimizer': {'values': ['adam']}}}


In [13]:
def s_optimizer(network, optimizer, learinin_rate):
    
    optimizer = optim.Adam(network.parameters(),lr = learinin_rate)

 
 return optimizer

IndentationError: unindent does not match any outer indentation level (<tokenize>, line 6)

## SweepCustomDataset

In [ ]:
class sweepCustomDataset(Dataset):

  
    def __init__(self, data, target=' C1'): # , normalize=True
        super(sweepCustomDataset, self).__init__()
        self.x = data.drop(target, 1)
        # # 데이터 표준화
        # if normalize:
        #scaler = StandardScaler()
        # self.x = pd.DataFrame(scaler.fit_transform(self.x))
        self.y = data[' C1']
        # 텐서 변환
        self.x = torch.tensor(self.x.values,requires_grad=True).float()
        self.y = torch.tensor(self.y.values,requires_grad=True).float()
        
    def __len__(self):
        return len(self.x)
    
    def __getitem__(self, idx):
        x = self.x[idx]
        y = self.y[idx]
        return x, y

## sweepload_data

In [ ]:
def sweepload_data(data,batch_size):
    

  dataset = sweepCustomDataset(data, ' C1')

  dataset_size = len(dataset)

  train_size = int(dataset_size * 0.8)

  # test_size = dataset_size - train_size 


  test_size = dataset_size - train_size 
  # 데이터 분할 
  train_dataset, test_dataset = random_split(dataset, [train_size, test_size])

  train_loader = DataLoader(train_dataset, 
                         batch_size= batch_size, 
                         shuffle=True,num_workers = 0)
    

  test_loader = DataLoader(test_dataset, 
                         batch_size= batch_size, 
                         shuffle=True,num_workers = 0)
  

  return train_loader , test_loader

## sweepNet

In [ ]:
class sweepNet(nn.Module):
  
    def __init__(self, num_features, dropout):
     
        super(sweepNet, self).__init__()
        
        # self.do1 = nn.Dropout(0.8)
        # self.fc1 = nn.Sequential(
            
        #     nn.Linear(num_features, 4,bias=True),
        #     nn.ReLU(),nn.Dropout(p =dropout) 
        # )
        # self.fc2 = nn.Sequential(
        #     nn.Linear(4, 2,bias=True),
        #     nn.ReLU(),nn.Dropout(p =dropout) 
        # )

        self.fc1 = nn.Linear(num_features, 1024)
        self.do1 = nn.Dropout(p =dropout)
        self.fc2 = nn.Linear(1024, 1)
        # self.do2 = nn.Dropout(p =dropout)
        # # 마지막 출력층의 Neuron은 1개로 설정
        
        # self.output = nn.Linear(512, 1)
        
        self.relu = torch.nn.LeakyReLU()
        self.bn1 = nn.BatchNorm1d(1024)
        # self.bn2 = nn.BatchNorm1d(512)
        # self.bn3 = nn.BatchNorm1d(32)



    def forward(self, x):
        
        x = self.fc1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.do1(x)

        
        x = self.fc2(x)
        # x = self.bn2(x)
        # x = self.relu(x)

        # x = self.do2(x)


        # x=  self.bn2(x)
        # x = F.relu(self.fc3(x))
        
        # x = self.output(x)
     

        # x = self.do1(x)
        # x1 = F.relu(self.fc1(x))
        # x2 = self.do2(x1)
        # x3 = F.relu(self.fc2(x2))
        # x4 = self.do3(x3)
        # x5 = self.output(x4)

        return x


## sweeptrain

In [ ]:
def sweeptrain(model, trainloader,criterion, optimizer, config, wandb):
    # gradient, topology와 관련된 정보를 visualization 하기 위한 코드
    
   
    
    wandb.watch(model, criterion, log="all", log_freq=10)
    example_ct = 0 
   
  
    for epoc in tqdm(range(config.epochs)):

      cumu_loss = 0
      model.train()
      

      for X, y in trainloader:

            X, y = X.to(device), y.to(device)
           
            

            optimizer.zero_grad()
           

            ## 1
            yhat = model(X)
            ## 2
            y = y.reshape(-1,1)
            loss = criterion(yhat,y)

            cumu_loss += loss.item()
            ## 3
            loss.backward()
            ## 4
            optimizer.step()

           

            example_ct +=  loss.item()

      avg_loss = cumu_loss / len(trainloader)




   

      #  visualization 하고 싶은 정보를 넘겨줄 수 있는 함수 
      wandb.log({"loss": cumu_loss}, step=epoc)
     
      

      print(f"TRAIN: EPOCH {epoc + 1:04d} / | Epoch LOSS {avg_loss:.4f}]")





## sweeptest

In [ ]:
def sweeptest(model, test_loader,wandb):
    
    y =[]
    hat=[]
    r=[]
    model.eval()
    
    with torch.no_grad():
        correct, total = 0, 0
        for x_test, y_test in test_loader:

            x_test, y_test = x_test.to(device), y_test.to(device)
           
            y.append(y_test)

            y_test_hat = model(x_test)

            
            y_shape = y_test.shape

            y_test_hat= y_test_hat.reshape(y_shape)

            hat.append(y_test_hat)

            r_square = r2score(y_test_hat,y_test)
            r.append(r_square)


            # pred = outputs.max(1, keepdim = True)[1]                                       
            # correct += pred.eq(y_test.view_as(pred)).sum().item() 

           
        print(f"Accuracy of the model on the {total} " +
              f"test x_test: {r_square}")
        


        # Make the plot
        # y_test= y_test.detach().numpy()

        # y_test_hat= y_test_hat.detach().numpy()

        y_test, y_test_hat = y_test.to(device_cpu), y_test_hat.to(device_cpu)
        fig, ax = plt.subplots()
        x_len = range(len(y_test))
        ax.plot(x_len, y_test, '*-')
        ax.plot(x_len, y_test_hat, 'o--')

        # Log the plot
        wandb.log({"plot": fig})
        fig
        
        # x_len = range(len(target))
        # plt.figure(figsize=(20,8))
        # plt.plot(x_len, target, 'o-')
        # plt.plot(x_len, preds, 'o--')

        # # Log the plot
        # wandb.log({"plot": fig})
        # wandb.finish()
        # fig
        

        
        wandb.log({"test_accuracy": r_square})

    #Save the model in the exchangeable ONNX format
    torch.onnx.export(model, x_test, "model.onnx")
    wandb.save("model.onnx")

    return

## sweeprun

In [ ]:
def sweeprun_ori(config = None):
 
    torch.manual_seed(42)
    

    wandb.init(config= config)
    w_config = wandb.config
  

    # wandb.config.update(config)
    
    data = pc2_4

    train_loader,test_loader = sweepload_data(data,w_config.batch_size)

    # test_loader = load_data_test(data,batch_size=config["test_batch"])

    model = sweepNet(NUM_FEATURES,w_config.dropout).to(device)

    criterion = nn.MSELoss().to(device)
    # optimizer = torch.optim.Adam(model.parameters(), lr=config["learning_rate"])

    optimizer= s_optimizer(model,w_config.optimizer, w_config.learning_rate)

    sweeptrain(model, train_loader,criterion, optimizer, w_config, wandb)

    sweeptest(model, test_loader,wandb)



    return model

# Run

In [ ]:
sweep_id = wandb.sweep(sweep_config, project="pc2_4_1024")

Create sweep with ID: sdcg841b
Sweep URL: https://wandb.ai/expect/pc2_4_1024/sweeps/sdcg841b


In [ ]:
wandb.agent(sweep_id, sweeprun_ori, count=100)

wandb: Agent Starting Run: jt3m22q2 with config:
wandb: 	batch_size: 480
wandb: 	dropout: 0.9
wandb: 	epochs: 30
wandb: 	learning_rate: 0.02281550753492334
wandb: 	optimizer: adam


<ipython-input-28-0cadbc44d34d>:6: FutureWarning:

In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only



  0%|          | 0/30 [00:00<?, ?it/s]

TRAIN: EPOCH 0001 / | Epoch LOSS 40.5476]
TRAIN: EPOCH 0002 / | Epoch LOSS 53.1307]
TRAIN: EPOCH 0003 / | Epoch LOSS 50.0029]
TRAIN: EPOCH 0004 / | Epoch LOSS 47.9306]
TRAIN: EPOCH 0005 / | Epoch LOSS 81.6038]
TRAIN: EPOCH 0006 / | Epoch LOSS 82.3333]
TRAIN: EPOCH 0007 / | Epoch LOSS 60.2229]
TRAIN: EPOCH 0008 / | Epoch LOSS 50.5253]
TRAIN: EPOCH 0009 / | Epoch LOSS 50.0550]
TRAIN: EPOCH 0010 / | Epoch LOSS 83.1995]
TRAIN: EPOCH 0011 / | Epoch LOSS 79.6706]
TRAIN: EPOCH 0012 / | Epoch LOSS 54.2018]
TRAIN: EPOCH 0013 / | Epoch LOSS 86.5481]
TRAIN: EPOCH 0014 / | Epoch LOSS 45.7032]
TRAIN: EPOCH 0015 / | Epoch LOSS 75.7095]
TRAIN: EPOCH 0016 / | Epoch LOSS 51.7150]
TRAIN: EPOCH 0017 / | Epoch LOSS 25.7616]
TRAIN: EPOCH 0018 / | Epoch LOSS 45.5729]
TRAIN: EPOCH 0019 / | Epoch LOSS 35.5855]
TRAIN: EPOCH 0020 / | Epoch LOSS 34.7180]
TRAIN: EPOCH 0021 / | Epoch LOSS 22.4174]
TRAIN: EPOCH 0022 / | Epoch LOSS 18.3754]
TRAIN: EPOCH 0023 / | Epoch LOSS 43.2768]
TRAIN: EPOCH 0024 / | Epoch LOSS 2

loss,▄▅▅▄██▆▅▅█▇▅█▄▇▅▂▄▃▃▂▂▄▂▄▂▂▁▂▁
test_accuracy,▁
loss,106.60828
test_accuracy,-0.16058


...

Run qa84e2hv errored: ValueError('Expected more than 1 value per channel when training, got input size torch.Size([1, 1024])')
wandb: ERROR Run qa84e2hv errored: ValueError('Expected more than 1 value per channel when training, got input size torch.Size([1, 1024])')
wandb: Agent Starting Run: 11yultfa with config:
wandb: 	batch_size: 240
wandb: 	dropout: 0.5
wandb: 	epochs: 30
wandb: 	learning_rate: 0.09908360774157966
wandb: 	optimizer: adam


  0%|          | 0/30 [00:00<?, ?it/s]

TRAIN: EPOCH 0001 / | Epoch LOSS 268.2955]
TRAIN: EPOCH 0002 / | Epoch LOSS 126.2296]
TRAIN: EPOCH 0003 / | Epoch LOSS 40.0156]
TRAIN: EPOCH 0004 / | Epoch LOSS 38.4840]
TRAIN: EPOCH 0005 / | Epoch LOSS 19.2086]
TRAIN: EPOCH 0006 / | Epoch LOSS 17.6734]
TRAIN: EPOCH 0007 / | Epoch LOSS 8.9388]
TRAIN: EPOCH 0008 / | Epoch LOSS 17.5146]
TRAIN: EPOCH 0009 / | Epoch LOSS 5.3211]
TRAIN: EPOCH 0010 / | Epoch LOSS 9.7729]
TRAIN: EPOCH 0011 / | Epoch LOSS 2.3313]
TRAIN: EPOCH 0012 / | Epoch LOSS 1.8491]
TRAIN: EPOCH 0013 / | Epoch LOSS 1.5291]
TRAIN: EPOCH 0014 / | Epoch LOSS 1.5121]
TRAIN: EPOCH 0015 / | Epoch LOSS 1.1042]
TRAIN: EPOCH 0016 / | Epoch LOSS 0.9057]
TRAIN: EPOCH 0017 / | Epoch LOSS 0.9769]
TRAIN: EPOCH 0018 / | Epoch LOSS 0.8034]
TRAIN: EPOCH 0019 / | Epoch LOSS 0.6121]
TRAIN: EPOCH 0020 / | Epoch LOSS 0.6274]
TRAIN: EPOCH 0021 / | Epoch LOSS 0.5398]
TRAIN: EPOCH 0022 / | Epoch LOSS 0.4819]
TRAIN: EPOCH 0023 / | Epoch LOSS 0.5234]
TRAIN: EPOCH 0024 / | Epoch LOSS 0.5084]
TRAIN: 

loss,█▄▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
test_accuracy,▁
loss,7.03112
test_accuracy,0.03134


wandb: Agent Starting Run: 9plhnko2 with config:
wandb: 	batch_size: 128
wandb: 	dropout: 0.6
wandb: 	epochs: 30
wandb: 	learning_rate: 0.08771021174553996
wandb: 	optimizer: adam
wandb: Ctrl + C detected. Stopping sweep.
